# How to connect to Azure OpenAI service via PromptSail and Langchain SDK

First import the OpenAI Python SDK and load your API key from the environment.


In [1]:
import os
from dotenv import load_dotenv, dotenv_values
from pprint import pprint


config = dotenv_values(".env")


azure_oai_key = config["AZURE_OPENAI_API_KEY"]
api_base_url = config["AZURE_OPENAI_ENDPOINT"]

deployment_name = config["AZURE_OPENAI_DEPLOYMENT_NAME"]
api_version = config["AZURE_OPENAI_API_VERSION"]

print(
    f"Azure OpenAI api key={azure_oai_key[0:3]}...{azure_oai_key[-5:]}"
)
print(
    f"Azure OpenAI api endpoint={api_base_url[0:17]}..."
)
print(f"Azure OpenAI deployment name={deployment_name[0:7]}...")
print(f"Azure OpenAI api version={api_version}")

Azure OpenAI api key=9b2...d67b3
Azure OpenAI api endpoint=https://openai-pr...
Azure OpenAI deployment name=gpt-35T...
Azure OpenAI api version=2023-07-01-preview


Test the direct connection to Azure OpenAI and Langchain SDK.


In [5]:

from langchain.chat_models import AzureChatOpenAI
#from langchain_community.chat_models import AzureChatOpenAI

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage


messages = [
    SystemMessage(
        content="You are a helpful assistant that help rewirte an jira ticket."
    ),
    HumanMessage(
        content="Give meaningful title to this bug, RuntimeError: CUDA out of memory. Tried to allocate X MiB (GPU X; X GiB total capacity; X GiB already allocated; X MiB free; X cached)"
    ),
]


In [21]:

chat = AzureChatOpenAI(
    openai_api_key=azure_oai_key,
    deployment_name=deployment_name,
    api_version=api_version,
    azure_endpoint=api_base_url,
    #base_url=api_base_url,
    #model=deployment_name,
)

In [22]:
chat(messages)

AIMessage(content='Bug: Insufficient GPU Memory Allocation Leads to RuntimeError')

## Create a request to the AzureOpenAI via PromptSail proxy

Run the docker and go to PromptSail UI http://promptsail.local/


You should add your own mappings in project settings. Create new project with you `project_slug`or edit existing one.

Set the `api_base` to your Azure OpenAI endpoint like
 
'https://**azure_openai_resource**.openai.azure.com/'
 
 and provider_name to Azure OpenAI.



```bash
{
    ai_providers: [
        {
            api_base: 'https://azure_openai_resource.openai.azure.com',
            provider_name: 'Azure OpenAI',
            ai_model_name: 'gpt-3.5-turbo'
        }
    ],
}
```

In this case we will use the project with project_slug -> 'project2'




In [29]:
ps_api_base = "http://promptsail.local:8000/project2/"


chat = AzureChatOpenAI(
    openai_api_key=azure_oai_key,
    deployment_name=deployment_name,
    api_version=api_version,
    azure_endpoint=ps_api_base,
    #base_url=api_base_url,
    #model=deployment_name,
)

In [30]:
chat(messages)

AIMessage(content='Bug: Out of Memory Error: CUDA\n')